In [8]:
import tensorflow as tf
import numpy as np

from vectorizer import fenToBitmap

In [93]:
X = list(map(lambda x: fenToBitmap(x), [
    "1r3rk1/p2qb1p1/Bp1pNn1p/2p5/2P5/2N3P1/PP2PP1P/R1BQ1RK1",
    "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR"
]))
y = [
    1242,
    -48
]

In [94]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [ ]:
# CREATE GENERATOR TO FEED TO MODEL FIT METHOD
# GENERATOR READ DATA LINE BY LINE AND RETURNS FEN/SCORE VALUE PAIRS

In [97]:
tf.keras.layers.Lambda(lambda x: fenToBitmap(x))(inputs)